# Calculate Q-Rookley

- Input data "20172022_processed_1_3_4.csv", from 2017/7/1 to 2022/12/31, same as used in Figlewski

- Use 500 rolling window for P density (when calculating EPK)

- Use logreturn instead of log+1

- Use 
```
r.source('Q_Rookley_main_3.R')
```
and
```
'EPK_library_1.r'
```
to calculate Q_Rookley, along with 
```
get_btc_prices_2015_2022
```
for physical density.

- Interest rate: 0

- Moneyness range is [min moneyness, max moneyness]

- Bandwidth 0.3

- tau=15

In [1]:
import platform
plat = platform.system()
import locale
if plat == 'Darwin':
    print('Using Macos Locale')
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
else:
    locale.setlocale(locale.LC_ALL, 'en_US.utf8')
import datetime
import pandas as pd
import sys
import matplotlib.pyplot as plt
import csv
import gc
import math
import numpy as np
import os
import pdb
import pickle
import statsmodels.api as sm
import time


import rpy2.robjects.packages as rpackages
from rpy2 import robjects

from scipy import stats
from scipy.integrate import simps
from scipy import interpolate

from src.helpers import save_dict, load_dict
from src.brc import BRC
from src.strategies import IronCondor
from src.quandl_data import get_btc_prices_2015_2022

from numpy.lib.function_base import insert

from statsmodels.nonparametric.kernel_regression import KernelReg as nadarajawatson

from sklearn.neighbors import KernelDensity

from pathlib import Path

def filter_sub(_df,  mindate, maxdate, tau):
    # Subset

    # Only calls, tau in [0, 0.25] and fix one day (bc looking at intra day here)
    # (_df['is_call'] == 1) & --> Dont need to filter for calls because that happens in the R script
    # Also need to consider put-call parity there
    sub = _df[(_df['date'] >= mindate) & (_df['date'] <= maxdate) & (_df['moneyness'] >= 0.5) & (_df['moneyness'] < 2) &(_df['mark_iv'] > 0)]# &

    if tau > 0:
        sub = sub[sub['tau'] == tau]

    nrows = sub.shape[0]
    if nrows == 0:
        raise(ValueError('Sub is empty'))
   
    sub['moneyness'] = round(sub['moneyness'], 3)
    sub['underlying_price'] = round(sub['underlying_price'], 2)

    sub = sub.drop_duplicates()
    
    print(sub.describe())

    if nrows > 50000:
        print('large df, subsetting')
        sub = sub.sample(10000)
        print(sub.describe())

    return sub

def prepare_confidence_band_data(df):

            # Old
                # Save for Confidence Band 
                    # Column 1 - Day
                    # Column 2 - Month
                    # Column 3 - Year
                    # Column 4 - Call Indicator
                    # Column 5 - Time Maturity in Days
                    # Column 6 - Strike
                    # Column 7 - IV
                    # Column 8 - Spot
                    # Column 9 - Risk Free Rate
                    # Column 10 - 
                    # Column 11 - Forward
                # 3	1	1997	0	49	3025	165.8	2861.706	0.031465	0.150026951	2863.26
                # Day Month Year Put Days Strike IV     Spot         Risk Free    Tau in Days (255)       Forward

    """
    # Mapping Table for Input

    # Column ... X
    # 1 - date
    # 2 - IV
    # 3 - 1 for Call, 0 for Put
    # 4 - Tau 
    # 5 - Strike
    # 6 - assigned meanwhile as put2call[,6]+ put2call[,7] -  put2call[,5]*exp(- mean(put2call[,8])* tau);
    # 7 - Spot
    # 8 - Probably Interest Rate
    # 9 - assigned meanwhile Moneyness: Spot/Strike
    """
    b = df[['date', 'mark_iv', 'putcall', 'tau', 'K', 'underlying_price', 'interest_rate', 'moneyness', 'instrument_price']]
    b['placeholder'] = 0
    b = b.reset_index(drop=True)
    # b['interest_rate'] = 0.00
    b['forward'] = 1

    # Reorder
    df_c = b[['date', 'mark_iv', 'putcall', 'tau', 'K', 'instrument_price', 'underlying_price', 'interest_rate', 'moneyness']]

    unique_maturities = df.days_to_maturity.unique()[0]
    fname = str('confidence_band_input_') + str(unique_maturities) + str('.csv')
    df_c.to_csv(fname, index = False)
    return fname

def bootstrap(conf_fname, phys_fname, rdate, tau, simmethod, r, out_dir, bw, lower, upper):
                            
    try:
        # MAKE SURE TO USE DIFFERENT 2ND INPUT FOR SP500!!
        moneyness, spd, epk, epk_lo, epk_up, logret, spd_logret, epk_logret, epk_logret_lo, epk_logret_up, epk_BS, tau, volas, spd_lo, spd_up, cdf_m, cdf_ret= r.bootstrap_epk(robjects.StrVector([conf_fname]), 
                                      robjects.StrVector([phys_fname]),
                                      robjects.StrVector(rdate),
                                      robjects.StrVector([out_dir]),
                                      robjects.FloatVector([bw]),
                                      robjects.FloatVector([lower]),
                                      robjects.FloatVector([upper])
                                     )
        # Can recover PD here as SPD/EPK
        spd_df = pd.DataFrame({'m': moneyness,
                               'spdy': spd,
                               'spdy_up': spd_up,
                               'spdy_dn': spd_lo,
                               'epk': epk,
                               'ret': logret,
                               'spd_ret': spd_logret,
                               'cb_ret_up': epk_logret_up,
                               'cb_ret_dn': epk_logret_lo,
                               'epk_ret': epk_logret,
                               'volatility': volas,
                               'epk_up': epk_up,
                               'epk_dn': epk_lo,
                               'cdf_m': cdf_m,
                               'cdf_ret': cdf_ret})

        return spd_df
    
    except Exception as e:
        print('Sim failed, proceeding with the next one... - ', simmethod)
        print('exception: ', e)
        
# Initiate R Objects
base = rpackages.importr("base") 
r = robjects.r
r.source('src/confidence_bands.R')
r.source('src/Q_Rookley_main_3.R')

# BTC prices
tdat = get_btc_prices_2015_2022()
tdat[::-1].to_csv('Data/BTC_USD_Quandl_2015_2022.csv', index = False)

bw_range = np.array([0.3])
r_bandwidth = bw_range[0]

simmethod = ['oldschool']

df = pd.read_csv('Data/20172022_processed_1_3_4.csv')
interest_rate_data = pd.read_csv('Data/IR_daily.csv')
interest_rate_data = interest_rate_data.rename(columns={'index': 'date','DTB3': 'interest_rate'})

df1=df[df['tau']==15]
dates=df1['date'].unique()
#df1=df[df['date']>=dates[9]]
#dates = df1['date'].unique()

dates1 = dates

error_i = []
for i0 in range(dates1.size):
    df2 = df1[df1['date']==dates1[i0]]
    
    curr_day = datetime.datetime.strptime(dates1[i0],'%Y-%m-%d')
    curr_day_starttime = curr_day.replace(hour = 0, minute = 0, second = 0, microsecond = 0)
    curr_day_endtime = curr_day.replace(hour = 23, minute = 59, second = 59, microsecond = 0)
    print(curr_day)

    df3 = pd.merge(df2, interest_rate_data, on='date', how='left')
    

    df3['interest_rate'] = 0
    df3['days_to_maturity'] = df3['tau']
    df3['tau'] = df3['tau']/365
    df3['underlying_price'] = df3['BTC_price']
    df3['instrument_price'] = df3['option_price']
    df3['putcall'][df3['putcall']=='C']=1
    df3['putcall'][df3['putcall']=='P']=0
    
    df_1 = df3[['date','BTC_price', 'K', 'interest_rate', 'tau', 'putcall', 'IV',  'moneyness',
                'days_to_maturity',  'underlying_price', 'instrument_price']]
    df_1['mark_iv']=df_1['IV']/100
    
    df_1.to_csv('Results/option_' + datetime.datetime.strftime(curr_day_starttime, '%Y%m%d') + '.csv')
    
    conf_fname = prepare_confidence_band_data(df_1)
    
    rdate = base.as_Date(curr_day.strftime('%Y-%m-%d'))
    rdate_f = base.format(rdate, '%Y-%m-%d')
    
    tau = df['tau'].unique()[0]
    
    spd_btc = bootstrap(conf_fname, 'Data/BTC_USD_Quandl_2015_2022.csv', rdate_f, tau, simmethod, r, 
                                     'Results/Rookley_Q_CB_1722_1_2_5_1_ttm15', r_bandwidth, df_1['moneyness'].min(), df_1['moneyness'].max())
    
    try :
        
        spd_btc.to_csv('Results/Rookley_Q_CB_1722_1_2_5_1_ttm15/btc_pk_'  + str(curr_day_starttime.__format__('%Y-%m-%d')) +'_bw_' + str(r_bandwidth) +  '.csv')
        
        print(spd_btc)
        
    except Exception as e:
        error_i.append(i0)

Using Macos Locale


/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
R[write to console]: 
Attaching package: ‘matlab’


R[write to console]: The following object is masked from ‘package:stats’:

    reshape


R[write to console]: The following objects are masked from ‘package:utils’:

    find, fix


R[write to console]: The following object is masked from ‘package:base’:

    sum


R[write to console]: KernSmooth 2.23 loaded
Copyright M. P. Wand 1997-2009

2017-07-13 00:00:00


/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_18556/1750272952.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_18556/1750272952.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_18556/1750272952.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] "bandwidth: 0.3"


...

[1] 44.694713  4.722982 66.258718
               [,1]      [,2]         [,3]        [,4]
sigmas -0.501254954 1.2886713 -2.509632778  3.76171273
       -0.492426540 1.2755158 -2.494361929  3.81841158
       -0.483675385 1.2624203 -2.478636364  3.87419678
       -0.475000149 1.2493880 -2.462461887  3.92906169
       -0.466399525 1.2364219 -2.445844396  3.98299983
       -0.457872243 1.2235250 -2.428789877  4.03600492
       -0.449417060 1.2107002 -2.411304405  4.08807087
       -0.441032769 1.1979505 -2.393394141  4.13919175
       -0.432718189 1.1852788 -2.375065330  4.18936182
       -0.424472173 1.1726879 -2.356324297  4.23857555
       -0.416293597 1.1601807 -2.337177449  4.28682758
       -0.408181368 1.1477599 -2.317631269  4.33411272
       -0.400134419 1.1354284 -2.297692312  4.38042600
       -0.392151706 1.1231888 -2.277367208  4.42576262
       -0.384232212 1.1110438 -2.256662656  4.47011796
       -0.376374945 1.0989960 -2.235585420  4.51348759
       -0.368578933 1.0870481 -

        0.333184493 0.7634626  0.928552584  1.33294935
        0.337026642 0.7686781  0.934218615  1.24493739
        0.340854085 0.7739166  0.939319526  1.15610155
        0.344666934 0.7791747  0.943850204  1.06644952
        0.348465301 0.7844491  0.947805633  0.97598932
        0.352249295 0.7897361  0.951180900  0.88472925
        0.356019024 0.7950324  0.953971196  0.79267796
        0.359774596 0.8003343  0.956171825  0.69984438
        0.363516116 0.8056382  0.957778207  0.60623782
        0.367243689 0.8109407  0.958785878  0.51186788
        0.370957419 0.8162381  0.959190501  0.41674453
        0.374657408 0.8215268  0.958987870  0.32087809
        0.378343758 0.8268031  0.958173911  0.22427923
        0.382016569 0.8320634  0.956744690  0.12695900
        0.385675939 0.8373039  0.954696416  0.02892880
        0.389321967 0.8425211  0.952025451 -0.06979957
        0.392954750 0.8477111  0.948728308 -0.16921394
        0.396574384 0.8528703  0.944801662 -0.26930174
        0.

/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_18556/1750272952.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_18556/1750272952.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_18556/1750272952.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] 44.694713  4.722982 66.258718
               [,1]      [,2]         [,3]     [,4]
sigmas -0.532273742 1.2150632 -2.393358341 2.866661
       -0.525225830 1.2051737 -2.382037520 2.903725
       -0.518227244 1.1953301 -2.370563325 2.940814
       -0.511277297 1.1855330 -2.358934855 2.977922
       -0.504375320 1.1757830 -2.347151246 3.015043
       -0.497520653 1.1660807 -2.335211672 3.052172
       -0.490712652 1.1564267 -2.323115340 3.089304
       -0.483950688 1.1468215 -2.310861497 3.126432
       -0.477234141 1.1372658 -2.298449426 3.163550
       -0.470562404 1.1277603 -2.285878447 3.200654
       -0.463934885 1.1183055 -2.273147918 3.237736
       -0.457351002 1.1089021 -2.260257233 3.274792
       -0.450810182 1.0995507 -2.247205825 3.311816
       -0.444311866 1.0902519 -2.233993165 3.348800
       -0.437855507 1.0810065 -2.220618762 3.385741
       -0.431440564 1.0718151 -2.207082162 3.422630
       -0.425066511 1.0626782 -2.193382952 3.459463
       -0.418732830 1.0535967 

        0.207893034 0.5904630  0.720765939 4.825860
        0.211261326 0.5935014  0.739307890 4.787067
        0.214618312 0.5966120  0.757609358 4.747331
        0.217964066 0.5997934  0.775664813 4.706650
        0.221298663 0.6030443  0.793468693 4.665017
        0.224622177 0.6063635  0.811015402 4.622429
        0.227934683 0.6097495  0.828299312 4.578881
        0.231236252 0.6132011  0.845314759 4.534369
        0.234526956 0.6167167  0.862056049 4.488888
        0.237806867 0.6202950  0.878517450 4.442434
        0.241076056 0.6239344  0.894693198 4.395002
        0.244334591 0.6276335  0.910577494 4.346588
        0.247582544 0.6313908  0.926164502 4.297186
        0.250819981 0.6352047  0.941448353 4.246793
        0.254046971 0.6390735  0.956423142 4.195404
        0.257263581 0.6429957  0.971082927 4.143015
        0.260469877 0.6469696  0.985421732 4.089620
        0.263665926 0.6509935  0.999433542 4.035215
        0.266851793 0.6550657  1.013112309 3.979795
        0.27

In [2]:
print(error_i)

[0, 1, 4, 5, 7, 10, 13, 19, 28, 29, 30, 34, 36, 37, 38, 39, 48, 92, 113]
